In [1]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *

In [20]:
import os
os.getcwd()

'/Users/reejungkim/Documents/Git/WebCrawler'

xattr -d com.apple.quarantine chromedriver

In [21]:
no_of_reviews = 10000

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

#https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(r"/Users/reejungkim/Documents/Git/WebCrawler/chrome96/chromedriver")
!which chromedriver

In [22]:
from selenium import webdriver
import chromedriver_autoinstaller


chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
#driver.get("http://www.python.org")
#assert "Python" in driver.title

In [23]:
wait = WebDriverWait( driver, 10 )

In [24]:
#Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.towneers.www&hl=ko&gl=US&showAllReviews=true"]

In [25]:
import pandas as pd
df = pd.DataFrame()

In [26]:
%%time

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    # open all reviews
    url = url+'&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,100):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')#scroll to load other reviews
        time.sleep(1)
    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    counter = 1
    for expand_page in expand_pages:
        try:
            
            author = str(expand_page.find("span", class_="X43Kjb").text)
            review_date = expand_page.find("span", class_="p2TkOb").text
            
            reviewer_ratings = expand_page.find("div", class_="jUL89d y92BAb").text
            #print("Reviewer Ratings: ", reviewer_ratings)

            review_body = str(expand_page.find("div", class_="UD7Dzf").text)
            #print("Review Body: ", review_body)
            
            star = expand_page.find("div", class_="pf5lIe").find_next()['aria-label']
            
            developer_reply = expand_page.find_parent().find("div", class_="LVQB0b")

            new_row = {'author':author, 'review_date':review_date, 
                       'reviewer_ratings':reviewer_ratings, 'review':review_body, 'star':star,
                       'developer_reply': developer_reply
                }
            df = df.append(new_row, ignore_index=True)

            counter+=1
        except:
            pass
driver.quit()

CPU times: user 1.67 s, sys: 67.3 ms, total: 1.74 s
Wall time: 1min 55s


In [27]:
import datetime as dt
timestamp = dt.datetime.today().strftime('%Y-%m-%d-%H:%m:%S')

In [28]:
os.chdir("/Users/reejungkim/Documents/Git/WebCrawler/output")

In [29]:
file_name = 'GooglePlayStore_Review_' + timestamp + '.csv'

In [30]:
df.to_csv(file_name, index = True)

In [31]:
driver.quit()